# Data Preparation Exercise Reference
------
## 基本作業

請讀取104年花東空品區三個測站的資料，進行分析，並回答以下問題

In [1]:
%matplotlib inline
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

首先，我們從檔案讀取資料，檢查一下資料的維度

In [2]:
hualian = pd.read_excel('../data/104年花蓮站_20160320.xls')
hualian.shape

(6205, 27)

In [3]:
taidong = pd.read_excel('../data/104年臺東站_20160320.xls')
taidong.shape

(6205, 27)

In [4]:
guanshan = pd.read_excel('../data/104年關山站_20160323.xls')
guanshan.shape

(5095, 27)

關山站的資料筆數明顯比另外兩個站少，讓我們檢查一下個測項的資料天數。

In [5]:
pd.crosstab(guanshan['測項'],'count')

col_0,count
測項,
AMB_TEMP,365
NO,360
NO2,360
NOx,360
O3,365
PM10,365
PM2.5,365
RAINFALL,365
RH,365


In [6]:
6205 == (365 * 3) + (5 * 3) + 5095

True

關山站少了三個測項，另外有三個測項少了5天的資料，如果補上這些資料，的確就跟花蓮、台東站一樣了。

## 資料清理

接下來，我們要把遺失值換成 np.nan，然後把雨量的 NR 換成 0。

In [7]:
def detect_epa_nan(x):
    ''' Search for missing value symbol and assign np.nan '''
    if re.findall('\#|\*|x', str(x))!=[]:
        return(np.nan)
    else:
        return(x)

def detect_epa_norain(x):
    ''' Replace 'NR' (no-rain) with 0 '''
    if str(x)=='NR':
        return(0)
    else:
        return(x)

def clean_epa_station(x):
    ''' Clean up a EPA station dataset '''
    # Rename columns
    col_names = ['date','station','item','h00','h01','h02','h03','h04','h05','h06','h07','h08','h09',
                'h10','h11','h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
    x.columns = col_names
    # Process NA and NR
    floatdata = x.iloc[:,3:]
    floatdata = floatdata.applymap(detect_epa_nan)
    floatdata = floatdata.applymap(detect_epa_norain)
    floatdata.astype(np.float32)
    x.iloc[:,3:] = floatdata
    # Done
    return(x)

hualian = clean_epa_station(hualian)
taidong = clean_epa_station(taidong)
guanshan = clean_epa_station(guanshan)

In [8]:
guanshan.head()

,date,station,item,h00,h01,h02,h03,h04,h05,h06,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,2015/01/01,關山,AMB_TEMP,13,13,12,13,14,13,13,...,16,15,14,13,13,12,12,12,12,12
1,2015/01/01,關山,NO,2.5,1.8,1.5,1.9,1.8,1.9,1.9,...,2,2,2.4,2.5,2.1,2,2.5,1.9,2,2.1
2,2015/01/01,關山,NO2,2.4,2.5,3,1.4,2.3,2.6,3.6,...,7.1,7,7.8,7.9,8.8,7.2,5.9,5.8,4.6,4
3,2015/01/01,關山,NOx,4.9,4.3,4.5,3.3,4.1,4.5,5.6,...,9.2,9,10,10,11,9.2,8.4,7.7,6.6,6.1
4,2015/01/01,關山,O3,33,33,33,35,38,33,30,...,43,44,43,44,41,41,40,38,38,37


## 資料格式轉換

如果我們想看各個測項單獨的狀態，以及彼此之間的關係，最佳的資料呈現方式，是將每個測項轉換成單獨的時間序列：

In [9]:
# Retrieve one item from EPA data and form a time series
def retrieve_epa_item(data, var):
    tmp = data.loc[data['item']==var,:]
    ts = pd.melt(tmp, id_vars=['date'], value_vars=tmp.keys()[3:], var_name='hour', value_name=var)
    ts[var] = ts[var].astype(np.float32)
    return(ts)

# Convert EPA dataset to a collection of time-series of items
def convert_epa_itemts(data):
    # All items
    items = list(set(data['item']))
    # Create the 1st dataframe
    newdata = retrieve_epa_item(data, items[0])
    # Loop through the rest of items
    for i in items[1:]:
        tmp = retrieve_epa_item(data, i)
        newdata = newdata.merge(tmp, on=['date','hour'], how='left')
    # Sort with date-hour to make the time-series in order
    newdata = newdata.sort_values(['date', 'hour'])
    # Done
    return(newdata)

# Do the conversion
hl = convert_epa_itemts(hualian)
td = convert_epa_itemts(taidong)
gs = convert_epa_itemts(guanshan)

如此一來，我們可以很快的計算某個單獨測項有多少個遺失值。

In [10]:
len(td) - td['O3'].count()

131

我們也可以用 [`DataFrame.apply`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) 一次計算每個測項的遺失值數量。

In [11]:
td.apply(lambda x: len(x)-x.count())

date            0
hour            0
PH_RAIN       499
CO            111
PM10          351
NO            145
RAIN_COND     499
NO2           145
RH             44
WD_HR          49
WS_HR          49
AMB_TEMP       44
O3            131
WIND_SPEED     40
SO2           185
NOx           145
WIND_DIREC     40
RAINFALL       67
PM2.5         314
dtype: int64

In [12]:
# 三個測站的測項不完全相同
print(hl.keys())
print(td.keys())
print(gs.keys())

Index(['date', 'hour', 'PH_RAIN', 'CO', 'PM10', 'NO', 'RAIN_COND', 'NO2', 'RH',
       'WD_HR', 'WS_HR', 'AMB_TEMP', 'O3', 'WIND_SPEED', 'SO2', 'NOx',
       'WIND_DIREC', 'RAINFALL', 'PM2.5'],
      dtype='object')
Index(['date', 'hour', 'PH_RAIN', 'CO', 'PM10', 'NO', 'RAIN_COND', 'NO2', 'RH',
       'WD_HR', 'WS_HR', 'AMB_TEMP', 'O3', 'WIND_SPEED', 'SO2', 'NOx',
       'WIND_DIREC', 'RAINFALL', 'PM2.5'],
      dtype='object')
Index(['date', 'hour', 'PM10', 'NO', 'NO2', 'RH', 'WD_HR', 'WS_HR', 'AMB_TEMP',
       'O3', 'WIND_SPEED', 'SO2', 'NOx', 'WIND_DIREC', 'RAINFALL', 'PM2.5'],
      dtype='object')


轉換成新格式後，我們可以利用 pandas.DataFrame.corr() 快速的計算同個測站不同測項的相關矩陣

In [13]:
td.iloc[:,2:].corr()

,PH_RAIN,CO,PM10,NO,RAIN_COND,NO2,RH,WD_HR,WS_HR,AMB_TEMP,O3,WIND_SPEED,SO2,NOx,WIND_DIREC,RAINFALL,PM2.5
PH_RAIN,1.000000,-0.042135,-0.135692,0.045226,0.273316,0.055999,0.333036,0.037688,-0.069249,-0.022882,-0.071950,-0.042609,-0.053129,0.061328,0.033734,0.523279,-0.117001
CO,-0.042135,1.000000,0.255145,0.399028,-0.022108,0.798970,-0.118078,-0.126034,-0.092150,-0.216774,0.245317,-0.011801,0.279279,0.789160,-0.124853,-0.058018,0.387693
PM10,-0.135692,0.255145,1.000000,0.024184,-0.006339,0.214265,-0.377530,-0.097221,0.173624,-0.018219,0.271322,0.241299,0.243439,0.184134,-0.097937,-0.083898,0.694217
NO,0.045226,0.399028,0.024184,1.000000,-0.008346,0.391307,-0.208761,-0.275049,0.103350,0.292727,-0.240725,0.099102,0.155515,0.651789,-0.296260,0.008334,-0.096615
RAIN_COND,0.273316,-0.022108,-0.006339,-0.008346,1.000000,-0.017785,0.067298,0.021278,0.083433,0.001025,-0.010941,0.036625,-0.024562,-0.017404,-0.007793,0.066045,-0.016527
NO2,0.055999,0.798970,0.214265,0.391307,-0.017785,1.000000,-0.059287,-0.058266,-0.210261,-0.098878,0.024707,-0.175576,0.218944,0.951786,-0.054946,-0.008224,0.279657
RH,0.333036,-0.118078,-0.377530,-0.208761,0.067298,-0.059287,1.000000,0.286957,-0.331417,-0.144073,-0.324560,-0.400335,-0.239348,-0.117143,0.303602,0.209090,-0.278670
WD_HR,0.037688,-0.126034,-0.097221,-0.275049,0.021278,-0.058266,0.286957,1.000000,-0.273042,-0.380335,-0.330594,-0.408732,-0.148383,-0.139066,0.829801,0.006320,0.025361
WS_HR,-0.069249,-0.092150,0.173624,0.103350,0.083433,-0.210261,-0.331417,-0.273042,1.000000,0.146847,0.208060,0.754290,0.079964,-0.139493,-0.281664,-0.062019,0.043594
AMB_TEMP,-0.022882,-0.216774,-0.018219,0.292727,0.001025,-0.098878,-0.144073,-0.380335,0.146847,1.000000,-0.297005,0.016945,-0.004275,0.015054,-0.378088,-0.011573,-0.254055


也可以計算不同測站品項之間的相關性：

In [14]:
pm25 = pd.DataFrame({'hualian':hl['PM2.5'], 'taidong':td['PM2.5'], 'guanshan':gs['PM2.5']})
pm25.corr()

,hualian,taidong,guanshan
hualian,1.000000,0.612239,0.586037
taidong,0.612239,1.000000,0.805572
guanshan,0.586037,0.805572,1.000000


把單一測項轉換成時間序列之後，也可以利用 `pandas.DataFrame.interpolate()` 做內插來取代遺失值。

In [15]:
td['NOx'].describe()

count    8615.000000
mean        7.484066
std         3.728248
min         0.900000
25%         4.700000
50%         7.100000
75%         9.500000
max        34.000000
Name: NOx, dtype: float64

In [16]:
tdo3_int = td['NOx'].interpolate()
tdo3_int.describe()

count    8760.000000
mean        7.504090
std         3.719959
min         0.900000
25%         4.700000
50%         7.200000
75%         9.500000
max        34.000000
Name: NOx, dtype: float64

在新的資料格式裡，我們也可以用 pandas.pivot_table() 來看測項之間的關係

In [17]:
tmp = td.loc[:,['PM2.5','RH','WIND_SPEED']]
tmp['RH'] = pd.cut(tmp['RH'], bins=[0., 25., 50., 75., 100.], labels=['<25','<50','<75','<100'])
tmp['WIND_SPEED'] = pd.qcut(tmp['WIND_SPEED'], 4)
tmp.head()
tmp.pivot_table(columns=['RH'])

RH,<50,<75,<100
PM2.5,12.427273,10.672629,7.333839


## 額外挑戰

## 在目錄下尋找特定檔案

前面的示範，都是處理少數檔案的情況，但是環保署的全部資料包含76個測站，雖然 copy-and-paste 76次也可以解決問題，但是 python 提供了更方便的工具 `os.walk()`，讓我們可以「遊走」指定資料夾底下的子目錄，然後用 `str.endswith()` 來尋找所有的 `.xls` 檔案。

In [18]:
import os

def find_xls_files(path):
    urls = []
    for root, dirs, files in os.walk(path):
        for fname in files:
            if(fname.endswith(".xls")):
                urls.append(os.path.join(root, fname))
    return(urls)

epafiles = find_xls_files('../data/')
epafiles[:5]

['../data/104年花蓮站_20160320.xls',
 '../data/104年臺東站_20160320.xls',
 '../data/104年關山站_20160323.xls']

我們可以透過這個方法，把所有測站的資料都一次做處理。

In [19]:
# Collect data from all files
data = []
nancounts = []
for f in epafiles:
    tmp = pd.read_excel(f)
    tmp = clean_epa_station(tmp)
    tmp = convert_epa_itemts(tmp)
    nancounts.append(tmp.apply(lambda x: len(x)-x.count()))
    data.append(tmp)

nancounts = pd.concat(nancounts, axis=1).T
nancounts.head()

/Users/tsyo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


,AMB_TEMP,CO,NO,NO2,NOx,O3,PH_RAIN,PM10,PM2.5,RAINFALL,RAIN_COND,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,date,hour
0,66.0,123.0,779.0,779.0,779.0,126.0,225.0,176.0,281.0,85.0,225.0,65.0,516.0,69.0,59.0,60.0,69.0,0.0,0.0
1,44.0,111.0,145.0,145.0,145.0,131.0,499.0,351.0,314.0,67.0,499.0,44.0,185.0,49.0,40.0,40.0,49.0,0.0,0.0
2,31.0,NaN,356.0,356.0,356.0,94.0,NaN,205.0,172.0,52.0,NaN,31.0,149.0,34.0,26.0,26.0,34.0,0.0,0.0


然後我們可以很快的計算每個測站、每個測項遺失值的數量。

In [20]:
nancounts['station'] =(epafiles)
nancounts.set_index('station', inplace=True)
nan_items = nancounts.apply(lambda x: len(x)-x.count())
nan_items

AMB_TEMP      0
CO            1
NO            0
NO2           0
NOx           0
O3            0
PH_RAIN       1
PM10          0
PM2.5         0
RAINFALL      0
RAIN_COND     1
RH            0
SO2           0
WD_HR         0
WIND_DIREC    0
WIND_SPEED    0
WS_HR         0
date          0
hour          0
dtype: int64

In [21]:
nan_stations = nancounts.T.apply(lambda x: len(x)-x.count())
nan_stations.head()

station
../data/104年花蓮站_20160320.xls    0
../data/104年臺東站_20160320.xls    0
../data/104年關山站_20160323.xls    3
dtype: int64

In [22]:
nan_stations.loc[nan_stations>6]

Series([], dtype: int64)

透過缺失值特徵的分析，我們可以思考如何建置比較長期的資料庫：我們希望有盡可能多的測站和測項，以及盡可能長的資料時間，透過合理的工具將資料補齊，以進一步位我們好奇的科學問題做出解答。